In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

In [2]:
es = Elasticsearch("https://elasticsearch.ceda.ac.uk:443")

In [4]:
fixes = {}

for item in scan(es, index="c3s-roocs-fix-prop"):
    fixes[item["_source"]["dataset_id"]] = item["_source"]["fixes"]

In [5]:
len(fixes.keys())

13960

In [12]:
ds_id = list(fixes.keys())[0]
ds_id

'c3s-cmip6.DCPP.EC-Earth-Consortium.EC-Earth3.dcppA-hindcast.s2014-r7i2p1f1.day.pr.gr.v20200508'

In [14]:
fix = fixes[ds_id]
fix

[{'fix': {'fix_id': 'VarAttrFix',
   'title': 'Apply Fix to Attributes of any Variable',
   'description': '\nEdits or adds the attributes of a given variables e.g. fixing standard name or adding missing standard name\n\nTakes a dictionary of fixes with each fix as a key and value pair with the attribute\nto be changed as the key and what the value should be as the value.\n\nFor example:\n  - inputs:\n    {"var_id": "lev",\n    "attrs":\n        {"long_name": "Dissolved Oxygen Concentration",\n        "standard_name": "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water"}\n    }\n',
   'category': 'attr_fixes',
   'reference_implementation': 'daops.data_utils.attr_utils.edit_var_attrs',
   'process_type': 'post_processor',
   'operands': {'var_id': 'time', 'attrs': {'long_name': 'valid_time'}},
   'source': {'name': 'ceda',
    'version': '',
    'comments': '',
    'url': 'https://github.com/cp4cds/c3s34g_master/tree/master/Decadal'}},
  'status': 'published',
  'timestamp':

In [30]:
from daops.data_utils.attr_utils import edit_var_attrs, edit_global_attrs, remove_coord_attr
from daops.data_utils.coord_utils import add_scalar_coord, add_coord
from daops.data_utils.var_utils import add_data_var

In [22]:
ds = None

In [23]:
# fix 1

operands = {'var_id': 'time', 'attrs': {'long_name': 'valid_time'}}
edit_var_attrs(ds_id, ds, **operands)

TypeError: 'NoneType' object is not subscriptable

In [24]:
# fix 2
operands = {
    'attrs': {
        'forcing_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: forcing_description',
        'physics_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: physics_description',
        'initialization_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: initialization_description',
        'startdate': 'derive: daops.fix_utils.decadal_utils.get_sub_experiment_id',
        'sub_experiment_id': 'derive: daops.fix_utils.decadal_utils.get_sub_experiment_id'}}

edit_global_attrs(ds_id, ds, **operands)


AttributeError: 'NoneType' object has no attribute 'attrs'

In [25]:
# fix 3

operands = {
    'var_id': 'reftime',
    'value': 'derive: daops.fix_utils.decadal_utils.get_reftime',
    'dtype': 'datetime64[ns]',
    'attrs': {
        'long_name': 'Start date of the forecast',
         'standard_name': 'forecast_reference_time'},
    'encoding': {'dtype': 'int32',
     'units': 'days since 1850-01-01',
     'calendar': 'derive: daops.fix_utils.decadal_utils.get_time_calendar'}}

add_scalar_coord(ds_id, ds, **operands)

AttributeError: 'NoneType' object has no attribute 'attrs'

In [27]:
# fix 4

operands = {'var_id': 'leadtime',
    'value': 'derive: daops.fix_utils.decadal_utils.get_lead_times',
    'dim': ['time'],
    'dtype': 'float64',
    'attrs': {'long_name': 'Time elapsed since the start of the forecast',
     'standard_name': 'forecast_period',
     'units': 'days'},
    'encoding': {'dtype': 'double'}}

add_coord(ds_id, ds, operands)

TypeError: add_coord() takes 2 positional arguments but 3 were given

In [29]:
# fix 5
operands = {
    'var_id': 'realization',
    'value': '1',
    'dtype': 'int32',
    'attrs': {'long_name': 'realization',
    'comment': 'For more information on the ripf, refer to the variant_label, initialization_description, physics_description and forcing_description global attributes'}}

add_data_var(ds_id, ds, **operands)


AttributeError: 'NoneType' object has no attribute 'assign'

In [31]:
# fix 6
operands = {'var_ids': 'derive: daops.fix_utils.decadal_utils.get_decadal_bnds_list'}

remove_coord_attr(ds_id, ds, **operands)

AttributeError: 'NoneType' object has no attribute 'cf'